# 单例模式

Singleton，即单实例模式，代表一个类仅仅只能拥有一个实例。表现在当 new 一个新的
instance 时，如果该类已经有一个实例存在，则返回那一个实例。

## 方法一：装饰器

优点：

* 比采用继承的方式更加直观

缺点：

* 尽管装饰后的类返回的就是同一个 Object，但是实际上装饰过以后，这个类就已经不是一个类，
而是一个函数。

In [20]:
def singleton(cls):
    instances = {}
    def getinstance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return getinstance

@singleton
class MyClass():
    pass

In [22]:
m = MyClass()
n = MyClass()
o = type(n)()

m is n and m is not o and n is not o

True

## 方法二：继承基类

通过在基类中覆写 `__new__()` 方法，来确保生成单个实例.

> 注意在这种方法中，实例虽然每次都是同一个实例，`__init__()` 方法每次都会被调用。

优点：

* 这个类是一个真实的类，不像装饰器一样

缺点：

* 多重继承的时候，__new__() 方法可能被重写

In [23]:
class Singleton(object):

    _instance = None

    def __new__(cls, *args, **kwargs):
        if not isinstance(cls._instance, cls):
            cls._instance = object.__new__(cls)
        return cls._instance

class MyClass(Singleton):
      pass

In [24]:
m = MyClass()
n = MyClass()

m is n

True

## 方法三：元类

优点：

* 真正的 class
* 继承不会出现问题

缺点： 无

In [25]:
class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

#Python2
class MyClass():
    __metaclass__ = Singleton

#Python3
class MyClass(metaclass=Singleton):
    pass

In [26]:
m = MyClass()
n = MyClass()

m is n

True

上面的代码不会每次调用 `__init__()` 方法，如果每次都要调用 `__init__()` 方法，可以在 `__call__()` 的 IF 语句中添加以下代码：

```python
else:
    cls._instances[cls].__init__(*args, **kwargs)
```

In [27]:
class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        else:
            cls._instances[cls].__init__(*args, **kwargs)
        return cls._instances[cls]

In [28]:
m = MyClass()
n = MyClass()

m is n

True

## 方法四：返回类的装饰器

优点：

* 真正的 Class
* 能够被继承

缺点：

* 可扩展性可能会有问题，因为每个类创建时都会同时创建另一个 Singleton 类。
* 在基类中调用 super() 时会出现问题，因为每层继承中间都穿插了一个 Singleton 类。

In [29]:
def singleton(cls):
    class class_w(cls):
        _instance = None
        def __new__(cls, *args, **kwargs):
            if class_w._instance is None:
                class_w._instance = object.__new__(cls)
                class_w._instance._sealed = False
            return class_w._instance
        def __init__(self, *args, **kwargs):
            if self._sealed:
                return
            super(class_w, self).__init__(*args, **kwargs)
            self._sealed = True
    class_w.__name__ = cls.__name__
    return class_w
        
@singleton
class MyClass():
    pass

In [30]:
m = MyClass()
n = MyClass()

m is n

True

## 总结

**最好的方法就是方法三，使用 metaclass 可以完美的实现 singleton 模式。**

## 参考

[Creating a singleton in Python](https://stackoverflow.com/q/6760685/12572033)